# Experiment with Motion Models

- This notebook contains code for experiment with motion models. The purpose of the notebook is to study the error of motion models, which are commonly used in research.
- In this study, we assume that the heading of the user is identical to the heading of the mobile phone and the heading values are computed from quaternions.
- In this study, we explore three step models and three stride-length models, which are commonly used in passive crowd-sourcing methods for WiFi radio map construction.
  - Step Models: local acceleration variance based, angular rate based, auto-correlation based.
  - Stride-Length Models: random, Weinberg, ZUPT

## Import

In [1]:
import sys
import os

# Add paths for resolve 
INDOOR_COMPETITION_20_DIR = os.path.join("..", "indoor-location-competition-20")
sys.path.append(INDOOR_COMPETITION_20_DIR)
CODE_DIR = ".."
sys.path.append(CODE_DIR)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from glob import glob
from tqdm import tqdm

from py_indoor_loc.model import read_data_file
from py_indoor_loc.floor_map import read_floor_data, extract_floor_map_geometries, scale
from py_indoor_loc.plot import plot_floor_map
from py_indoor_loc.sensors import compute_earth_acce_heading, compute_earth_acce_heading_ahrs
from py_indoor_loc.pdr.common import compute_step_heading, compute_rel_positions
from py_indoor_loc.pdr.common import RelPositionPredictor, compute_cumulative_step_positions
from py_indoor_loc.pdr.step_detection import AcfSDModel, LocalAcceVarianceSDModel, AngularRateSDModel
from py_indoor_loc.pdr.stride_length import WeinbergSLModel, ZUPTSLModel, RandomSLModel

In [16]:
from py_indoor_loc.pdr.stride_length import AbcSLModel

In [3]:
from compute_f import compute_step_positions

In [4]:
np.random.seed(19)

In [5]:
def compute_path_length(waypoint):
  return np.linalg.norm(waypoint[1:, 1:] - waypoint[:-1, 1:], axis=1).sum()


def compute_path_duration_secs(waypoint):
  return (waypoint[-1, 0] - waypoint[0, 0]) / 1000

In [6]:
def compute_error_metrics(actual_step_positions: np.ndarray, 
                          pred_step_positions: np.ndarray,
                          duration_step_size: int = 10) -> dict:
                          
  actual_path_length = compute_path_length(actual_step_positions[:, 1:])
  predicted_path_length = compute_path_length(pred_step_positions[:, 1:])

  actual_path_rel_timestamps = actual_step_positions[:, 0] - actual_step_positions[0, 0]
  pred_path_rel_timestamps = pred_step_positions[:, 0] - pred_step_positions[0, 0]

  n_segments = np.ceil(actual_path_rel_timestamps[-1] / duration_step_size / 1000).astype(np.int64)

  segments = []
  segment_ae = []
  segment_ape = []
  for k in range(n_segments):
    segments.append((k + 1) * duration_step_size)
    pl_true = compute_path_length(actual_step_positions[actual_path_rel_timestamps < (k + 1) * duration_step_size * 1000, 1:])
    pl_pred = compute_path_length(pred_step_positions[pred_path_rel_timestamps < (k + 1) * duration_step_size * 1000, 1:])
    segment_ae.append(abs(pl_pred - pl_true))
    segment_ape.append(abs(pl_pred - pl_true) / (pl_true + 1e-6))

  return {
    "actual_distance": actual_path_length,
    "predicted_distance": predicted_path_length,
    "distance_error": abs(predicted_path_length - actual_path_length),
    "distance_error_pct": abs((predicted_path_length - actual_path_length) / (actual_path_length + 1e-6)),
    "segments": segments,
    "segment_distance_error": segment_ae,
    "segment_distance_error_pct": segment_ape,
  }

## Experiment Setup

### Track Selection

* In this experiment, we only consider

In [7]:
TRAIN_DATA_DIR = "../../data/train/"

In [8]:
def list_tracks(site_id: str, floor_id: str, data_dir: str) -> list[str]:
  track_floor_path = os.path.join(data_dir, site_id, floor_id)
  return list(glob(track_floor_path + "/*.txt"))

* Compute length and duration of tracks

In [9]:
track_list = []

for site_path in glob(TRAIN_DATA_DIR + "/*"):
  site_id = os.path.basename(site_path)

  for floor_path in glob(site_path + "/*"):
    floor_id = os.path.basename(floor_path)
    
    for track_path in glob(floor_path + "/*.txt"):
      track_id = os.path.basename(track_path)[:-len(".txt")]
      
      track_list.append((site_id, floor_id, track_id))

In [10]:
len(track_list)

26925

In [9]:
count = 0

track_stats = []

for (site_id, floor_id, track_id) in tqdm(track_list):
  track_path = os.path.join(TRAIN_DATA_DIR, site_id, floor_id, track_id + ".txt")
  try:
    path_data_collection = read_data_file(track_path)
    duration = compute_path_duration_secs(path_data_collection.waypoint)
    track_stats.append({
      "site_id": site_id,
      "floor_id": floor_id,
      "track_id": track_id,
      "duration": compute_path_duration_secs(path_data_collection.waypoint),
      "length": compute_path_length(path_data_collection.waypoint),
    })

    if duration > 60:
      count += 1

    if count >= 30000:
      break
  
  except Exception as ignored:
    pass

 19%|█▉        | 5133/26925 [06:02<28:22, 12.80it/s]  

Failed to processing line: ['1559699299331', 'TYPE_ROTATION_VECTOR', '0.17401376', '-0.0031055238']. Caused by list index out of range


100%|██████████| 26925/26925 [42:10<00:00, 10.64it/s]  


In [11]:
track_stats_df = pd.DataFrame(track_stats)
track_stats_df.to_csv("../../data/track_stats.csv", index=False)

NameError: name 'track_stats' is not defined

In [12]:
track_stats_df = pd.read_csv("../../data/track_stats.csv")

In [13]:
track_stats_df = track_stats_df[track_stats_df["length"] >= 50].reset_index(drop=True)

### Model Setup

* **Hyper-parameter tuning**

In [14]:
def run_experiment(site_floor_tracks, model):

  experiment_results = []

  for site_id, floor_id, track_id in site_floor_tracks:
    track_path = os.path.join(TRAIN_DATA_DIR, site_id, floor_id, track_id + ".txt")

    try:
      path_data_collection = read_data_file(track_path)
      actual_step_positions = compute_step_positions(path_data_collection.acce, path_data_collection.ahrs, path_data_collection.waypoint)

      pred_rel_positions = model.predict(path_data_collection)
      pred_step_positions = compute_cumulative_step_positions(pred_rel_positions)
      result = compute_error_metrics(actual_step_positions, pred_step_positions)
      result.update({
        "site_id": site_id,
        "floor_id": floor_id,
        "track_id": track_id,
        "model_name": model.name,
      })
      experiment_results.append(result)

    except Exception as e:
      print(f"Error: {type(e)}: {str(e)}")
  
  return experiment_results

In [15]:
def model_selection(params, models, site_floor_tracks):

  min_mape = float("inf")
  best_param = None
  best_model = None

  for param, model in tqdm(list(zip(params, models))):
    try:
      experiment_results = run_experiment(site_floor_tracks, model)
      experiment_results = pd.DataFrame(experiment_results)

      mape = experiment_results["distance_error_pct"].mean()

      if mape < min_mape:
        best_param = param
        best_model = model
        min_mape = mape
    except Exception as ignored:
      pass
    
  return best_param, best_model

In [37]:
def create_zupt_sl_model(step_detector, **kwargs):
  return ZUPTSLModel(step_detector, **kwargs)

In [17]:
from py_indoor_loc.model import PathDataCollection

class FixedSLModel(AbcSLModel):

  def __init__(self, stride_length) -> None:
    super().__init__()
    self._stride_length = stride_length

  @property
  def stride_length(self):
    return self._stride_length

  def predict(self, path_data_collection: PathDataCollection) -> np.ndarray:
    n = len(path_data_collection.acce)
    return np.ones(n, dtype=np.float32) * self._stride_length
    
  def returns_at_step(self) -> bool:
    return False

In [18]:
fixed_sl_model = FixedSLModel(0.8)

* Finding the optimal hyper-parameters for angular rate sd model

In [22]:
site_floor_track_model_selection = track_stats_df.sample(frac=0.05)[["site_id", "floor_id", "track_id"]].values

In [24]:
len(site_floor_track_model_selection)

340

In [19]:
params = []
test_models = []

for median_filter_window_size in [4, 8, 16]:
  for stance_threshold in [0.1, 0.2, 0.4, 0.6, 0.8, 1.0]:
    sd_model = AngularRateSDModel(median_filter_window_size=median_filter_window_size, stance_threshold=stance_threshold)
    params.append((median_filter_window_size, stance_threshold))
    test_models.append(
      RelPositionPredictor(
        sd_model=sd_model,
        sl_model=fixed_sl_model,
        name="angular_rate + fixed_sl")
    )

In [25]:
best_param, best_model = model_selection(params, test_models, site_floor_track_model_selection[:10])

  0%|          | 0/18 [00:00<?, ?it/s]

Error: <class 'AssertionError'>: 


  6%|▌         | 1/18 [00:05<01:25,  5.01s/it]

Error: <class 'AssertionError'>: 


 33%|███▎      | 6/18 [00:29<00:58,  4.88s/it]

...

 67%|██████▋   | 12/18 [00:58<00:29,  4.86s/it]

Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 


 72%|███████▏  | 13/18 [01:03<00:24,  4.82s/it]

Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 


100%|██████████| 18/18 [01:27<00:00,  4.86s/it]


In [26]:
best_param

(8, 0.4)

In [27]:
best_result = run_experiment(site_floor_track_model_selection, best_model)

In [28]:
best_result = pd.DataFrame(best_result)

In [31]:
best_result["distance_error_pct"].mean(), best_result["distance_error_pct"].std()

(0.32321050284769853, 0.24300587479262326)

* Finding the optimal hyper-parameters for local acceleration variance model

In [32]:
params = []
test_models = []

for window_size in [4, 8, 16]:
  for swing_threshold in [1, 2, 3, 4]:
    for stance_threshold in np.arange(0.5, swing_threshold / 2 + 1e-6, 0.5):
      sd_model = LocalAcceVarianceSDModel(window_size=window_size, swing_threshold=swing_threshold, stance_threshold=stance_threshold, use_ahrs=True)
      params.append((window_size, swing_threshold, stance_threshold))
      test_models.append(
        RelPositionPredictor(
          sd_model=sd_model,
          sl_model=fixed_sl_model,
          name="local_acc_variance + fixed_sl")
      )

In [33]:
best_param, best_model = model_selection(params, test_models, site_floor_track_model_selection[:10])

 10%|█         | 3/30 [00:18<02:42,  6.02s/it]

Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 


 13%|█▎        | 4/30 [00:24<02:34,  5.96s/it]

Error: <class 'AssertionError'>: 


 20%|██        | 6/30 [00:36<02:24,  6.00s/it]

...

 93%|█████████▎| 28/30 [02:49<00:12,  6.11s/it]

Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 


 97%|█████████▋| 29/30 [02:55<00:06,  6.08s/it]

Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 


100%|██████████| 30/30 [03:01<00:00,  6.04s/it]


In [34]:
best_param

(4, 2, 1.0)

In [75]:
df = pd.DataFrame(run_experiment(site_floor_track_model_selection[:20], best_model))

Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 
Error: <class 'AssertionError'>: 


In [38]:
local_acce_var_step_detector = LocalAcceVarianceSDModel(window_size=4, swing_threshold=2, stance_threshold=1, use_ahrs=True)
acf_step_detector = AcfSDModel(t_min=40, t_max=100, use_ahrs=True)
angular_rate_step_detector = AngularRateSDModel(median_filter_window_size=8, stance_threshold=0.4)

weinberg_sl_model = WeinbergSLModel(window_size=16, use_ahrs=True)
random_sl_model = RandomSLModel(base_sl=1.0, noise_pct=0.15)

zupt_local_acce_var_sl_model = create_zupt_sl_model(local_acce_var_step_detector, window_size=4, fs=50, use_ahrs=True)
zupt_acf_sl_model = create_zupt_sl_model(acf_step_detector, window_size=4, fs=50, use_ahrs=True)
zupt_angular_rate_sl_model = create_zupt_sl_model(angular_rate_step_detector, window_size=4, fs=50, use_ahrs=True)

In [39]:
acf_weinberg = RelPositionPredictor(sd_model=acf_step_detector,
                                    sl_model=weinberg_sl_model,
                                    name="acf + weinberg")

local_acce_var_weinberg = RelPositionPredictor(
    sd_model=local_acce_var_step_detector,
    sl_model=weinberg_sl_model,
    name="local_acce_var + weinberg")

angular_rate_weinberg = RelPositionPredictor(
    sd_model=angular_rate_step_detector,
    sl_model=weinberg_sl_model,
    name="angular_rate + weinberg")

acf_random = RelPositionPredictor(sd_model=acf_step_detector,
                                  sl_model=random_sl_model,
                                  name="acf + random")

local_acce_var_random = RelPositionPredictor(
    sd_model=local_acce_var_step_detector,
    sl_model=random_sl_model,
    name="local_acce_var + random")

angular_rate_random = RelPositionPredictor(sd_model=angular_rate_step_detector,
                                           sl_model=random_sl_model,
                                           name="angular_rate + random")

acf_zupt = RelPositionPredictor(sd_model=zupt_acf_sl_model.step_detection_model,
                                sl_model=zupt_acf_sl_model,
                                name="acf + zupt")

local_acce_var_zupt = RelPositionPredictor(
    sd_model=zupt_local_acce_var_sl_model.step_detection_model,
    sl_model=zupt_local_acce_var_sl_model,
    name="local_acce_var + zupt")

angular_rate_zupt = RelPositionPredictor(
    sd_model=zupt_angular_rate_sl_model.step_detection_model,
    sl_model=zupt_angular_rate_sl_model,
    name="angular_rate + zupt")

models = [
  acf_weinberg,
  local_acce_var_weinberg,
  angular_rate_weinberg,
  acf_random,
  local_acce_var_random,
  angular_rate_random,
  acf_zupt,
  local_acce_var_zupt,
  angular_rate_zupt
]

## Running Experiment

In [40]:
from compute_f import compute_step_positions

In [43]:
site_floor_tracks = track_stats_df[["site_id", "floor_id", "track_id"]].values.tolist()

In [45]:
experiment_results = []

for site_id, floor_id, track_id in tqdm(site_floor_tracks[:1]):
  track_path = os.path.join(TRAIN_DATA_DIR, site_id, floor_id, track_id + ".txt")

  try:
    path_data_collection = read_data_file(track_path)
    actual_step_positions = compute_step_positions(path_data_collection.acce, path_data_collection.ahrs, path_data_collection.waypoint)

    for model in models:
      pred_rel_positions = model.predict(path_data_collection)
      pred_step_positions = compute_cumulative_step_positions(pred_rel_positions)
      result = compute_error_metrics(actual_step_positions, pred_step_positions)
      result.update({
        "site_id": site_id,
        "floor_id": floor_id,
        "track_id": track_id,
        "model_name": model.name,
      })
      experiment_results.append(result)

  except Exception as e:
    print(f"Error: {type(e)}: {str(e)}")

100%|██████████| 1/1 [00:03<00:00,  3.23s/it]


In [46]:
pd.DataFrame(experiment_results).to_csv("../../data/Results_PDR_20231010.csv", sep=";")

In [47]:
df = pd.read_csv("../../data/Results_PDR_20231010.csv", sep=";", index_col=0)

In [48]:
df[(df["actual_distance"] >= 10) & (df["actual_distance"] <= 100)].groupby("model_name").agg({"distance_error_pct": "median"})

,distance_error_pct
model_name,
acf + random,1.186621
acf + weinberg,0.035446
acf + zupt,0.230943
angular_rate + random,0.903521
angular_rate + weinberg,0.025556
angular_rate + zupt,3.658154
local_acce_var + random,0.532653
local_acce_var + weinberg,0.742868
local_acce_var + zupt,39.264539
